<a href="https://colab.research.google.com/github/NNLogic/NNPosterPublic/blob/main/%5BNNPoster%5DJson_Images_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明 / Description
PDF/PNG/JPGからNNPoster用のJSON画像ファイルを作成するスクリプトになります。

Betaテスト版です。

★[1]→[2]→[3]の順番に実行してください。

※NNPosterに同梱されているUnityのスクリプトでは、crunch圧縮によりファイルサイズを小さくできます。

The script to create a JSON image file for NNPoster from PDF/PNG/JPG:

"This is a beta test version.

★Please execute the steps in the following order: [1] → [2] → [3].

※The Unity script included in NNPoster allows for file size reduction through crunch compression.

In [ ]:
# @title #変換設定 / Settings
#@markdown ↓ファイル名を表示する場合、未入力にしてください。/  If you want to display the file name, please leave it blank.
rename = "" #@param {type:"string"}
#@markdown ↓ファイル名の末尾番号をJsonの画像位置にする場合、"off"にしてください。/  If you want to use the file name's trailing number for the image position in the Json, set "off".
renumber = "on" #@param ["on", "off"]
#@markdown ↓スライドを変換する場合、max_widthは1024を推奨しています。/ When converting slides, we recommend using a max_width of 1024."
max_width = '2048' #@param ["1024", "2048", "4096", "8192", "Unlimited"]


#変換 / Convert

・Googleドライブからファイルをコピーする場合は、[0]を事前に実行してください。/ To copy a file from Google Drive, please execute [0] beforehand.


※ファイルサイズが大きい場合、Googleドライブから変換することを推奨します。/ If the file size is large, it is recommended to convert it from Google Drive.

In [ ]:
#@title [0] Googleドライブをマウントする / Mounting Google Drive
%cd  /content

import ipywidgets as widgets
from google.colab import drive

if 'drive' in locals():
  drive.flush_and_unmount()

drive.mount('/content/drive')

In [ ]:
#@title [1]　ファイルを読込 / Loading files
%cd  /content

load_type = "PDF" #@param ["PDF", "PNG, JPG"]
load_from = "Local Computer" #@param ["Google Drive", "Local Computer"]

#@markdown ↓Google Driveの場合、指定してください。/ In the case of Google Drive, please specify.
#@markdown 　PDFはファイルパス、PNG/JPGはフォルダパス / For PDF files, provide the file path, and for PNG/JPG files, provide the folder path.
load_path = "/content/drive/MyDrive/" #@param {type:"string"}

if 'rename' not in locals():
    rename = "Image"
if 'max_width' not in locals():
    max_width = 1024

from pkgutil import extend_path
from google.colab import files
import os
import shutil

!rm -f *.pdf *.png *.jpg *.json

def get_last_folder_path(load_path):
    load_path = load_path.rstrip(os.path.sep)
    dir_path, folder_name = os.path.split(load_path)
    return os.path.basename(folder_name)

if load_type ==  "PNG, JPG":
  image_files = []
  if load_from == "Local Computer":
    upload_files = files.upload()
    image_files = [file for file in upload_files if file.endswith('.png') or file.endswith('.jpg')]

  if load_from == "Google Drive":
    if not os.path.exists(load_path):
      print("Error:load_path is invalid")
      raise SystemExit
    copy_dest = "/content"
    os.chdir(load_path)
    output_file_name =  get_last_folder_path(load_path)

    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]
    no=0
    for file in image_files:
      no = no + 1
      src_path = os.path.join(load_path, file)
      dest_path = os.path.join(copy_dest, file)
      shutil.copy(src_path, dest_path)
    os.chdir("/content")
    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]

  no=0
  for file in image_files:
    no = no + 1
    if rename != "":
      file_name, file_extension = os.path.splitext(file)
      new_name =  rename+"-"+str(no)+file_extension
      os.rename(file, os.path.join(os.path.dirname(file), new_name))

if load_type ==  "PDF":
  pdf_file_name =""
  base_file_name =""

  if load_from == "Local Computer":
    result = files.upload()
    pdf_file_name = list(result.keys())[0]
    base_file_name = os.path.splitext(pdf_file_name)[0]

  if load_from == "Google Drive":
    if not os.path.exists(load_path):
      print("Error:load_path is invalid")
      raise SystemExit
    copy_dest = "/content"  # コピー先のディレクトリパスを指定してください
    pdf_file_name = os.path.basename(load_path)
    base_file_name = os.path.splitext(pdf_file_name)[0]
    output_file_name = base_file_name
    dest_path = os.path.join(copy_dest, pdf_file_name)
    shutil.copy(load_path, dest_path)

  !apt-get install -y poppler-utils

  output_name = base_file_name if rename == "" else rename;

  if max_width == 'Unlimited':
    !pdftoppm -png "$pdf_file_name" "$output_name"
  else:
    !pdftoppm -scale-to $max_width -png "$pdf_file_name"  "$output_name"



In [ ]:
#@title [2]JSONファイルへの変換 / Converting to JSON file
%cd  /content

!sudo apt-get install imagemagick
!pip install wand

import os, sys
from wand.image import Image
import datetime
import base64
import json
import re

if 'max_width' not in locals():
    max_width = "1024"
if 'rename' not in locals():
    rename = "Image"
if 'renumber' not in locals():
    renumber = "on"

no = 0
images = []
files = os.listdir("./")
def extract_number(file_name):
    match = re.search(r"\d+$", os.path.splitext(file_name)[0])
    return int(match.group()) if match else 0

sorted_files = sorted(files, key=extract_number)
for file_name in sorted_files:
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
      im = Image(filename=file_name)

      if max_width != "Unlimited":
        if im.width > int(max_width):
          target_width = int(max_width)
          target_height = int(im.height * (target_width / im.width))
          im.resize(target_width, target_height)

      im.flip()
      im.convert("RGBA")
      im.options['dds:mipmaps'] = "0"
      im.compression = "dxt1"
      im.format = 'dds'
      b64_txt = base64.b64encode(im.make_blob()[128:])

      type_str = "10" if im.compression == "dxt1" else "10" # type DXT1 10 DXT5 10

      if renumber == "off":
        match = re.search(r"\d+$", os.path.splitext(file_name)[0])
        no = int(match.group()) if match else no + 1
        no = 200 if no > 200 else 1 if no < 1 else no
      else:
         no = no + 1

      image_dict = {"no":str(no),"name":os.path.splitext(file_name)[0],"type":type_str,"width":str(im.width),"height":str( im.height),"base64":b64_txt.decode('utf-8')};
      images.append(image_dict)

dict= {"version":"001","images":images}

# output
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")

if 'output_file_name' not in locals():
    output_file_name  = formatted_datetime

json_file_name = output_file_name + '.json'
with open(json_file_name, "w") as outputFile:
  json.dump(dict, outputFile, indent=2 )

In [ ]:
#@title [3]ファイルのダウンロード / Downloading the file
from google.colab import files

files.download(json_file_name)